## Imports

In [1]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
from tabula.io import read_pdf
import re
import os

## Import data from PDF

In [2]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.read_pdf('condo.pdf', pages='all',lattice=True)

## Pandas Manipulation

In [3]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [4]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [5]:
df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only
1,Prive North\r5000 Island Estates Dr 1408\rAven...,3/31/2022,7/27/2022,85,"$5,000,000.00",3913,"$1,277.79",Bento Queiroz,"Compass Florida, LLC.",Viviane Wolak,One Sotheby's INT'L Realty,Prive North 5000 Island Estates Dr 1408 Aventura,Prive North,5000 Island Estates Dr 1408 Aventura
2,Reach Condo\r68 SE 6th St 4111\rMiami,3/15/2022,7/29/2022,99,"$3,200,000.00",2526,"$1,266.83",Juan Carlos De Campos,Miami New Realty,Elen Oliveira,One World Properties,Reach Condo 68 SE 6th St 4111 Miami,Reach Condo,68 SE 6th St 4111 Miami
3,Bella Mare\r6000 Island Blvd 1501\rAventura,4/5/2022,7/25/2022,26,"$3,000,000.00",4109,$730.10,Brandee Goldstein,"FIP Realty Services, LLC",Ronit Shiro,Beachfront Realty Inc,Bella Mare 6000 Island Blvd 1501 Aventura,Bella Mare,6000 Island Blvd 1501 Aventura
4,Three Tequesta\r848 Brickell Key Dr 4603\rMiami,3/1/2022,7/25/2022,89,"$3,000,000.00",3506,$855.68,Berenice Elguezabal,Coldwell Banker Realty,Susana Ojeda,Florida Capital Realty,Three Tequesta 848 Brickell Key Dr 4603 Miami,Three Tequesta,848 Brickell Key Dr 4603 Miami
5,Jade Ocean\r17121 Collins Ave 4005\rSunny Isle...,7/11/2018,7/25/2022,1453,"$2,570,000.00",1933,"$1,329.54",Wanda Shortt,Yaffe International Realty,Paula Sanchez,Unique International Properties,Jade Ocean 17121 Collins Ave 4005 Sunny Isles ...,Jade Ocean,17121 Collins Ave 4005 Sunny Isles Beach
6,Acqualina Ocean Residence\r17875 Collins Ave 3...,4/4/2022,7/29/2022,32,"$2,550,000.00",2214,"$1,151.76",Viviane Wolak,One Sotheby's INT'L Realty,Daniel Dubin,"Related ISG Realty, LLC.",Acqualina Ocean Residence 17875 Collins Ave 39...,Acqualina Ocean Residence,17875 Collins Ave 3902 Sunny Isles Beach
7,Fontainebleau II Condo\r4401 Collins Ave 2702/...,4/21/2022,7/27/2022,50,"$2,360,000.00",1496,"$1,577.54",Christian Dreyfuss PA,Brown Harris Stevens,Michele Redlich,Coldwell Banker Realty,Fontainebleau II Condo 4401 Collins Ave 2702/2...,Fontainebleau II Condo,4401 Collins Ave 2702/2704 Miami Beach
8,400 Sunny Isles\r400 Sunny Isles Bch Blvd PH20...,4/15/2022,7/28/2022,95,"$2,050,000.00",3140,$652.87,Ryan Aulenti,"FIP Realty Services, LLC",Ruben Campbell,Luxury Real Estate Group LLC,400 Sunny Isles 400 Sunny Isles Bch Blvd PH200...,,400 Sunny Isles 400 Sunny Isles Bch Blvd PH200...
9,Santa Maria Condo\r1643 Brickell Ave 2505\rMiami,5/4/2022,7/26/2022,65,"$2,100,000.00",2580,$813.95,Antonio Mebarak,Fortune International Realty,Silvia Coltrane,The Corcoran Group,Santa Maria Condo 1643 Brickell Ave 2505 Miami,Santa Maria Condo,1643 Brickell Ave 2505 Miami
10,T0urnberry Ocean Colony\r16047 Collins Ave 320...,3/22/2022,7/25/2022,98,"$2,050,000.00",2245,$913.14,David Copelow,Terrabella Realty,Olga Mirer,"Atlantic VIP Realty Group, LLC",T0urnberry Ocean Colony 16047 Collins Ave 3202...,T,0urnberry Ocean Colony 16047 Collins Ave 3202 ...


In [6]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only'],
      dtype='object')

## Geocoder Setup and Execution

In [7]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [8]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [9]:
df['loc']

1     (5000 Island Estates Dr #1408, Aventura, FL 33...
2     (68 SE 6th St Apt 4111, Miami, FL 33131, USA, ...
3     (6000 Island Blvd #1501, Aventura, FL 33160, U...
4     (848 Brickell Key Dr APT 4603, Miami, FL 33131...
5     (17121 Collins Ave APT 4005, Sunny Isles Beach...
6     (17875 Collins Ave #3902, Sunny Isles Beach, F...
7     (4401 Collins Ave, Miami Beach, FL 33140, USA,...
8     (400 Sunny Isles Blvd, Sunny Isles Beach, FL 3...
9     (Santa Maria, 1643 Brickell Ave APT 2505, Miam...
10    (16047 Collins Ave APT 3202, Sunny Isles Beach...
Name: loc, dtype: object

In [10]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [11]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,Prive North\r5000 Island Estates Dr 1408\rAven...,3/31/2022,7/27/2022,85,"$5,000,000.00",3913,"$1,277.79",Bento Queiroz,"Compass Florida, LLC.",Viviane Wolak,One Sotheby's INT'L Realty,Prive North 5000 Island Estates Dr 1408 Aventura,Prive North,5000 Island Estates Dr 1408 Aventura,"(5000 Island Estates Dr #1408, Aventura, FL 33...","(25.9470842, -80.1306458, 0.0)",25.947084,-80.130646,0.0
2,Reach Condo\r68 SE 6th St 4111\rMiami,3/15/2022,7/29/2022,99,"$3,200,000.00",2526,"$1,266.83",Juan Carlos De Campos,Miami New Realty,Elen Oliveira,One World Properties,Reach Condo 68 SE 6th St 4111 Miami,Reach Condo,68 SE 6th St 4111 Miami,"(68 SE 6th St Apt 4111, Miami, FL 33131, USA, ...","(25.7680813, -80.1926812, 0.0)",25.768081,-80.192681,0.0
3,Bella Mare\r6000 Island Blvd 1501\rAventura,4/5/2022,7/25/2022,26,"$3,000,000.00",4109,$730.10,Brandee Goldstein,"FIP Realty Services, LLC",Ronit Shiro,Beachfront Realty Inc,Bella Mare 6000 Island Blvd 1501 Aventura,Bella Mare,6000 Island Blvd 1501 Aventura,"(6000 Island Blvd #1501, Aventura, FL 33160, U...","(25.9427925, -80.13540019999999, 0.0)",25.942792,-80.135400,0.0
4,Three Tequesta\r848 Brickell Key Dr 4603\rMiami,3/1/2022,7/25/2022,89,"$3,000,000.00",3506,$855.68,Berenice Elguezabal,Coldwell Banker Realty,Susana Ojeda,Florida Capital Realty,Three Tequesta 848 Brickell Key Dr 4603 Miami,Three Tequesta,848 Brickell Key Dr 4603 Miami,"(848 Brickell Key Dr APT 4603, Miami, FL 33131...","(25.7693285, -80.1837717, 0.0)",25.769329,-80.183772,0.0
5,Jade Ocean\r17121 Collins Ave 4005\rSunny Isle...,7/11/2018,7/25/2022,1453,"$2,570,000.00",1933,"$1,329.54",Wanda Shortt,Yaffe International Realty,Paula Sanchez,Unique International Properties,Jade Ocean 17121 Collins Ave 4005 Sunny Isles ...,Jade Ocean,17121 Collins Ave 4005 Sunny Isles Beach,"(17121 Collins Ave APT 4005, Sunny Isles Beach...","(25.9343529, -80.1210992, 0.0)",25.934353,-80.121099,0.0
6,Acqualina Ocean Residence\r17875 Collins Ave 3...,4/4/2022,7/29/2022,32,"$2,550,000.00",2214,"$1,151.76",Viviane Wolak,One Sotheby's INT'L Realty,Daniel Dubin,"Related ISG Realty, LLC.",Acqualina Ocean Residence 17875 Collins Ave 39...,Acqualina Ocean Residence,17875 Collins Ave 3902 Sunny Isles Beach,"(17875 Collins Ave #3902, Sunny Isles Beach, F...","(25.940858, -80.1208766, 0.0)",25.940858,-80.120877,0.0
7,Fontainebleau II Condo\r4401 Collins Ave 2702/...,4/21/2022,7/27/2022,50,"$2,360,000.00",1496,"$1,577.54",Christian Dreyfuss PA,Brown Harris Stevens,Michele Redlich,Coldwell Banker Realty,Fontainebleau II Condo 4401 Collins Ave 2702/2...,Fontainebleau II Condo,4401 Collins Ave 2702/2704 Miami Beach,"(4401 Collins Ave, Miami Beach, FL 33140, USA,...","(25.8170079, -80.12229529999999, 0.0)",25.817008,-80.122295,0.0
8,400 Sunny Isles\r400 Sunny Isles Bch Blvd PH20...,4/15/2022,7/28/2022,95,"$2,050,000.00",3140,$652.87,Ryan Aulenti,"FIP Realty Services, LLC",Ruben Campbell,Luxury Real Estate Group LLC,400 Sunny Isles 400 Sunny Isles Bch Blvd PH200...,,400 Sunny Isles 400 Sunny Isles Bch Blvd PH200...,"(400 Sunny Isles Blvd, Sunny Isles Beach, FL 3...","(25.9291821, -80.12832829999999, 0.0)",25.929182,-80.128328,0.0
9,Santa Maria Condo\r1643 Brickell Ave 2505\rMiami,5/4/2022,7/26/2022,65,"$2,100,000.00",2580,$813.95,Antonio Mebarak,Fortune International Realty,Silvia Coltrane,The Corcoran Group,Santa Maria Condo 1643 Brickell Ave 2505 Miami,Santa Maria Condo,1643 Brickell Ave 2505 Miami,"(Santa Maria, 1643 Brickell Ave APT 2505, Miam...","(25.755259, -80.1947467, 0.0)",25.755259,-80.194747,0.0
10,T0urnberry Ocean Colony\r16047 Collins Ave 320...,3/22/2022,7/25/2022,98,"$2,050,000.00",2245,$913.14,David Copelow,Terrabella Realty,Olga Mirer,"Atlantic VIP Realty Group, LLC",T0urnberry Ocean Colony 16047 Collins Ave 3202...,T,0urnberry Ocean Colony 1

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

## Correction Section
To be used if a name is off. Format for making corrections is as follows:

`df.at[index_num,'column_name']=('what you want inserted')`

In [12]:
df.at[8,'building_name']=('400 Sunny Isles')
df.at[8,'address_only']=('400 Sunny Isles Beach Boulevard PH 2008')

df.at[10,'building_name']=('Turnberry Ocean Colony')
df.at[10,'address_only']=('16047 Collins Ave 3202')

In [13]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('July 24th - July 30rd')

# marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['price_per_sqft'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [14]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [15]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [16]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condoSalesWeekEnding07_30_22
